In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from dotenv import load_dotenv
import os

# === 1️⃣ 加载环境变量 (.env 文件) ===
load_dotenv()

# === 2️⃣ 从 .env 读取配置 ===
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_CHAT_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")
AZURE_EMBED_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBED_DEPLOYMENT")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")


In [2]:
# === 3️⃣ 加载 PDF 文档 ===
loader = PyPDFLoader("./白话机器学习的数学_图灵图书_立石贤吾_Z_Library.pdf")
docs = loader.load()

In [3]:
# === 4️⃣ 切块 ===
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = splitter.split_documents(docs)

In [4]:
# === 5️⃣ 向量化（Azure Embedding）===
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=AZURE_EMBED_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)
vectorstore = FAISS.from_documents(chunks, embeddings)

# === 6️⃣ 构建问答链（Azure Chat 模型）===
llm = AzureChatOpenAI(
    azure_deployment=AZURE_CHAT_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)
qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

# === 7️⃣ 提问 ===
query = "根据文档，美绪是谁，有什么特色，学习到了什么？"
print("Q:", query)
print("A:", qa.invoke(query))


Q: 根据文档，美绪是谁，有什么特色，学习到了什么？
A: {'query': '根据文档，美绪是谁，有什么特色，学习到了什么？', 'result': '美绪是绫乃的朋友，从大学时就认识。她的大学专业是计算机视觉，并在研究中用到过机器学习。美绪的特色是不会拒绝绫乃的请求，性格友好，并且也喜欢吃甜食。在交流中，她向绫乃分享了关于机器学习的学习经历，尽管她指出机器学习中确实有很多数学表达式，但基础部分是可以逐步理解的。'}
